**PANASHE W MADZUDZO R204568J HCS12**

In [ ]:
pip install tensorflow

     |████████████████████████████████| 462 kB 7.4 MB/s 


In [ ]:
pip install keras

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.models import load_model
import tensorflow as tf

In [ ]:
#create the base pretrained model
my_model = InceptionV3(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

96124928/96112376 [==============================] - 1s 0us/step


In [ ]:
my_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#Testing the model on a single frame
animal_hog = tf.io.read_file('/content/drive/MyDrive/Colab Notebooks/Machine Learning/ML ASSIGNMENT/pig.jpg')
animal_hog= tf.io.decode_image(animal_hog,channels=3)
animal_hog= tf.image.resize(animal_hog,[299,299])
animal_hog = tf.expand_dims(animal_hog, axis=0)
animal_hog = tf.keras.applications.inception_v3.preprocess_input(animal_hog)

In [ ]:
#Predicting and Decoding the Predictions
predictions = tf.keras.applications.inception_v3.decode_predictions(my_model.predict(animal_hog), top=1)

49152/35363 [=========================================] - 0s 0us/step


In [ ]:
#Extracting the top prediction only
for (i, (imagenetID, label, prob)) in enumerate(predictions[0]):
    print("{}. {}: {:.2f}%".format(i+1, label, prob*100))

1. hog: 94.04%


In [ ]:
for (i,(imagenetID, label, prob)) in enumerate(predictions[0]):
    print(label.lower())

hog


In [ ]:
filename = "/content/drive/MyDrive/Colab Notebooks/Machine Learning/ML ASSIGNMENT/model.h5"

In [ ]:
my_model.save(filename)

In [ ]:
!pip install -q streamlit

     |████████████████████████████████| 10.1 MB 8.4 MB/s 
     |████████████████████████████████| 164 kB 64.2 MB/s 
     |████████████████████████████████| 111 kB 74.9 MB/s 
     |████████████████████████████████| 76 kB 6.6 MB/s 
     |████████████████████████████████| 4.3 MB 60.8 MB/s 
     |████████████████████████████████| 181 kB 54.6 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 131 kB 63.1 MB/s 
     |████████████████████████████████| 428 kB 70.2 MB/s 
     |████████████████████████████████| 130 kB 75.3 MB/s 
     |████████████████████████████████| 793 kB 69.4 MB/s 
     |████████████████████████████████| 381 kB 64.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1

In [ ]:
!pip -q install pyngrok

     |████████████████████████████████| 745 kB 8.5 MB/s 


In [ ]:
from pyngrok import ngrok

In [ ]:
!ngrok authtoken 27hDyVmRWxXefkwe6hYDnl3FxGh_cNoWhjWpLT9Bi9NY8Ej9

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
#PANASHE W MADZUDZO R204568J HCS212
%%writefile app.py
import streamlit as st
import cv2
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.models import load_model
import tensorflow as tf

from keras.models import load_model
import sys

#Loading the Inception model
filename = "/content/drive/MyDrive/Colab Notebooks/Machine Learning/ML ASSIGNMENT/model.h5"
model= load_model(filename,compile=(False))



#Functions
def predict(frame, model):
    # Pre-process the image for model prediction
    img = cv2.resize(frame, (299, 299))
    img = img.astype(np.float32)
    img = np.expand_dims(img, axis=0)

    img /= 255.0

    # Predict with the Inceptionv3 model
    prediction = model.predict(img)

    # Convert the prediction into text
    pred_text = tf.keras.applications.inception_v3.decode_predictions(prediction, top=1)
    for (i, (imagenetID, label, prob)) in enumerate(pred_text[0]):
        label  = ("{}: {:.2f}%".format(label, prob * 100))

    st.markdown(label)

def predict2(frame, model):
    # Pre-process the image for model prediction
    img = cv2.resize(frame, (299, 299))
    img = img.astype(np.float32)
    img = np.expand_dims(img, axis=0)

    img /= 255.0

    # Predict with the Inceptionv3 model
    prediction = model.predict(img)

    # Convert the prediction into text
    pred_text = tf.keras.applications.inception_v3.decode_predictions(prediction, top=1)
    for (i, (imagenetID, label, prob)) in enumerate(pred_text[0]):
        pred_class = label
       

    return pred_class

def object_detection(search_key,frame, model):
    label = predict2(frame,model)
    label = label.lower()
    if label.find(search_key) > -1:
        st.image(frame, caption=label)

        return label, frame
    else:
        pass
        #st.text('Not Found')
        #return sys.exit()

# Main App
def main():
    """Deployment"""
    st.title("OBJECT DETECTION SYSTEM")
    st.text("Built with Streamlit and Inceptionv3")

    activities = ["Detect Objects", "About"]
    choice = st.sidebar.selectbox("Choose Activity", activities)
    
    if choice == "Detect Objects":
        st.subheader("Upload the video to detect objects")

        video_file = st.file_uploader("upload the video...", type=["mp4", "avi"])

        if video_file is not None:
            path = video_file.name
            with open(path,mode='wb') as f: 
                f.write(video_file.read())         
                st.success("Saved Video")
                video_file = open(path, "rb").read()
                st.video(video_file)
            cap = cv2.VideoCapture(path)
            frame_width = int(cap.get(3))
            frame_height = int(cap.get(4))

            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            output = cv2.VideoWriter('/content/drive/MyDrive/Colab Notebooks/Machine Learning/ML ASSIGNMENT/output.mp4', fourcc, 20.0, (frame_width, frame_height))
            
            if st.button("Detect Video Objects"):
                
                # Start the video prediction loop
                while cap.isOpened():
                    ret, frame = cap.read()
    
                    if not ret:
                        break
    
                    # Perform object detection
                    #frame = object_detection(frame, model)
                    predict(frame, model)
    
                    # Display the resulting frame
                    #st.image(frame, caption='Video Stream', use_column_width=True)
    
                cap.release()
                output.release()
                cv2.destroyAllWindows()
                
            key = st.text_input('Search key')
            key = key.lower()
            
            if key is not None:
            
                if st.button("Search for the objects in the video"):
                    
                    
                    # Start the video prediction loop
                    while cap.isOpened():
                        ret, frame = cap.read()
        
                        if not ret:
                            break
        
                        # Perform object detection
                        object_detection(key,frame, model)
                        #frame = predict(frame, model)
        
                        # Display the resulting frame
                        #st.image(frame, caption='Video Stream', use_column_width=True)
        
                    cap.release()
                    output.release()
                    cv2.destroyAllWindows()

    elif choice == "About":
        st.subheader("By")
        st.text("PANASHE W MADZUDZO R204568J HCS212")


if __name__ == '__main__':
    main()


Writing app.py


In [ ]:
#!streamlit --server.maxUploadSize 2

In [ ]:
!streamlit run --server.port 80 app.py&>/dev/null& 

In [ ]:
#!streamlit run app.py --server.maxUploadSize 2 port 80 &>/dev/null& 

In [ ]:
address = ngrok.connect(port=8501)

In [ ]:
address

<NgrokTunnel: "http://a14d-35-233-184-78.ngrok.io" -> "http://localhost:80">

In [ ]:
#ngrok.kill()